In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import numpy as np
import  pandas as pd

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,log_loss
from sklearn.model_selection import train_test_split
from pandas_ml import ConfusionMatrix as cm
from sklearn.preprocessing import StandardScaler
from sklearn import svm

from sklearn.neighbors.kde import KernelDensity

from IPython.display import display_html

from scipy.stats import norm

import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st
import statsmodels as sm
import matplotlib

from IPython.display import display,clear_output,HTML

In [2]:
#### Source= https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side

def mydisplay(dfs, names=[]):
    html_str = ''
    if names:
        html_str += ('<tr>' + 
                     ''.join(f'<td style="text-align:center">{name}</td>' for name in names) + 
                     '</tr>')
    html_str += ('<tr>' + 
                 ''.join(f'<td style="vertical-align:top"> {df.to_html(index=False)}</td>' 
                         for df in dfs) + 
                 '</tr>')
    html_str = f'<table>{html_str}</table>'
    html_str = html_str.replace('table','table style="display:inline"')
    display_html(html_str, raw=True)

In [3]:
train=pd.read_csv('Feature_Vector_50_50_12_24_random_state_0.csv',index_col='User')
print('Infected/Unifected Users')
display(train.Target.value_counts())
user_Dec=train.index.values

Infected/Unifected Users


1    2910
0    2910
Name: Target, dtype: int64

In [4]:
jan=pd.read_csv('Feature-Set_Jan.csv',index_col='User')
print('Infected/Unifected Users')
display(jan.Target.value_counts())
user_jan=jan.index.values

Infected/Unifected Users


0    51123
1      764
Name: Target, dtype: int64

In [5]:
no=[user for user in user_jan if user not in user_Dec]

o=[user for user in user_jan if user in user_Dec]

jan.drop(o,inplace=True)

print("After eliminating overlapping users:")
print(jan.Target.value_counts())

After eliminating overlapping users:
0    46596
1      532
Name: Target, dtype: int64


#### There are 232 Malicious and 4527 Non-malicious Users in January 2018, who overlap with users in Sept-Dec 2017
#### The number of users after balancing the data in January 2018

In [6]:
x=jan[jan.Target==1]
y=jan[jan.Target==0]
y=y.sample(n=532,random_state=10)
z=x.append(y)
z=z.sample(frac=1,random_state=10)
display(z.Target.value_counts())

1    532
0    532
Name: Target, dtype: int64

### Non-parametric approach

In [7]:
Y_test=z.Target.values

In [8]:
lst=[]
for col in train.columns[:-1]:
    ### Non-Malicious Users
    X_train=train[train.Target==0][col].values
    Y_train=train[train.Target==0]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
    clf.fit(scaled_X)
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    non_mal_lst=clf.predict(X_test)
    
    ### Malicious users
    X_train=train[train.Target==1][col].values
    Y_train=train[train.Target==1]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
    clf.fit(scaled_X)
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    mal_lst=clf.predict(X_test)
    df=pd.DataFrame({'Non_Mal':non_mal_lst.reshape(1,-1)[0].tolist(),'Mal':mal_lst.reshape(1,-1)[0].tolist(),
              'Ground_Truth':Y_test.reshape(1,-1)[0].tolist()},index=z.index.values)

    df['Non_Mal']=df['Non_Mal'].apply(lambda x:'Non_Malicious' if x==1 else 'Malicious')
    df['Mal']=df['Mal'].apply(lambda x:'Malicious' if x==1 else 'Non_Malicious')
    df['Ground_Truth']=df['Ground_Truth'].apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')
    print('\033[1m' + str(col) + '\033[0m\n')
    mydisplay((pd.crosstab(df.Non_Mal,df.Ground_Truth),pd.crosstab(df.Mal,df.Ground_Truth)),names=['Non-Malicious','Malicious'])
    print("-----------------------------------------------------------------------------------------------------")
    file_name='Non-parametric/Non_para_col_'+str(col)+'.csv'
    df.to_csv(file_name)

SP



Non-Malicious,Malicious
Malicious,Non_Malicious
94,23
438,509
Malicious,Non_Malicious
410,106
122,426


-----------------------------------------------------------------------------------------------------
DP



Non-Malicious,Malicious
Malicious,Non_Malicious
56,68
476,464
Malicious,Non_Malicious
241,47
291,485


-----------------------------------------------------------------------------------------------------
NAPP



...

-----------------------------------------------------------------------------------------------------
2200



Non-Malicious,Malicious
Malicious,Non_Malicious
120,7
412,525
Malicious,Non_Malicious
444,158
88,374


-----------------------------------------------------------------------------------------------------
2300



Non-Malicious,Malicious
Malicious,Non_Malicious
115,6
417,526
Malicious,Non_Malicious
432,152
100,380


-----------------------------------------------------------------------------------------------------


#### Non-Parametric Test w/kernelDensity 

https://jakevdp.github.io/PythonDataScienceHandbook/05.13-kernel-density-estimation.html

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html#sklearn.neighbors.KernelDensity

In [18]:
for col in train.columns[:-1]:
    X_train=train[train.Target==0][col].values
    Y_train=train[train.Target==0]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(scaled_X.reshape(-1,1))
    X_test=scaler.transform(z['SP'].values.reshape(-1,1))
    non_mal=np.exp(kde.score_samples(X_test.reshape(-1,1)))
    
    X_train=train[train.Target==1][col].values
    Y_train=train[train.Target==1]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(scaled_X.reshape(-1,1))
    X_test=scaler.transform(z['SP'].values.reshape(-1,1))
    mal=np.exp(kde.score_samples(X_test.reshape(-1,1)))
    
    df=pd.DataFrame({'Non_Mal':non_mal,'Mal':mal,'Ground_Truth':Y_test.reshape(1,-1)[0]})
    df['Factor']=df['Non_Mal']/df['Mal']
    df['Factor']=df['Factor'].apply(lambda x: 'Non_Mal' if x>1 else 'Mal')
    df['Ground_Truth']=df['Ground_Truth'].apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')
    
    print('\033[1m' + str(col) + '\033[0m\n')
    display(pd.crosstab(df.Ground_Truth,df.Factor))
    print("-----------------------------------------------------------------------------------------------------")
    print("-----------------------------------------------------------------------------------------------------")
    file_name='Non-parametric/KDE/Non_para_col_'+str(col)+'.csv'
    df.to_csv(file_name)

SP



Factor,Mal,Non_Mal
Ground_Truth,,
Malicious,294,238
Non_Malicious,30,502


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
DP



Factor,Mal
Ground_Truth,
Malicious,532
Non_Malicious,532


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
NAPP



...

-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
2200



Factor,Mal,Non_Mal
Ground_Truth,,
Malicious,529,3
Non_Malicious,372,160


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
2300



Factor,Mal,Non_Mal
Ground_Truth,,
Malicious,528,4
Non_Malicious,361,171


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
